### PACKAGES

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from time import time
import numpy as np
from sklearn.utils import shuffle
os.chdir('C:\\Users\\A Sua\\Documents\\FIU\\CAP2020')

### DEFINE FUNCTIONS

#### timeit wrapper

In [2]:
def timing(f):
    def wrap(*args):
        time1 = time()
        ret = f(*args)
        time2 = time()
        print('{:s} function took {:.3f} ms'.format(f.__name__, (time2-time1)*1000.0))

        return ret
    return wrap

In [3]:
@timing
def run_func():
    print('hi')
run_func()

hi
run_func function took 0.997 ms


### IMPORT DATA

In [4]:
@timing
def import_data():
    
    #converter = lambda x: pd.to_numeric(x, 'coerce')
    #df = pd.read_csv('data\\raw\\PENN_SP500_daily.csv', delim_whitespace=True, converters={1: converter}, header=None)
    df = pd.read_csv('data\\raw\\PENN_SP500_daily.csv')
    #df = shuffle(df, random_state=0)
    df = df[['date',
    'TICKER',
    'COMNAM',
    'BIDLO',
    'ASKHI',
    'PRC',
    'VOL',
    'RET',
    #'BID',
    #'ASK',
    'SHROUT',
    'OPENPRC']]

    return df
    
df = import_data()
df.columns = ['date', 'symbol', 'name', 'bidlo', 'askhi', 'price', 'vol', 'ret', 
              #'bid', 'ask', 
              'sout', 'open']
print('initial shape {}'.format(df.shape))

df.head()

C:\PROGRA1\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3326: DtypeWarning: Columns (5,6,9,19,48,56) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


import_data function took 19650.774 ms
initial shape (2337279, 10)


,date,symbol,name,bidlo,askhi,price,vol,ret,sout,open
0,20000103,ORCL,ORACLE CORP,111.6250,125.1875,118.1250,24831819.0,0.054099,1423672.0,124.62500
1,20000104,ORCL,ORACLE CORP,105.0000,118.6250,107.6875,29568565.0,-0.08836,1423672.0,115.50000
2,20000105,ORCL,ORACLE CORP,96.0000,106.3750,102.0000,42032160.0,-0.052815,1423672.0,101.62500
3,20000106,ORCL,ORACLE CORP,94.6875,105.0000,96.0000,27848885.0,-0.058824,1423672.0,100.15625
4,20000107,ORCL,ORACLE CORP,93.5625,103.5000,103.3750,23218034.0,0.076823,1423672.0,95.00000


### BEGIN CLEANING

### DROP DUPLICATES

In [5]:
print('old shape {}'.format(df.shape))
df = df.drop_duplicates()
print('new shape: {}'.format(df.shape))

old shape (2337279, 10)
new shape: (2337166, 10)


### DROP ROWS WITH WRONG VALUES

In [6]:
"""MESSY DATA"""
indices_to_drop = df[df['ret']=='B'].index
df = df.drop(index=indices_to_drop)

indices_to_drop = df[df['ret']=='C'].index
df = df.drop(index=indices_to_drop)
#print('{} rows were dropped'.format(len(indices_to_drop)))

### STANDARDIZE, FORMALIZE COLUMNS (make into proper datatype)

#### CHANGE date to DATETIME

In [7]:
df['date'] = pd.to_datetime(df['date'], format='%Y%m%d', errors='ignore')
df.head()

,date,symbol,name,bidlo,askhi,price,vol,ret,sout,open
0,2000-01-03,ORCL,ORACLE CORP,111.6250,125.1875,118.1250,24831819.0,0.054099,1423672.0,124.62500
1,2000-01-04,ORCL,ORACLE CORP,105.0000,118.6250,107.6875,29568565.0,-0.08836,1423672.0,115.50000
2,2000-01-05,ORCL,ORACLE CORP,96.0000,106.3750,102.0000,42032160.0,-0.052815,1423672.0,101.62500
3,2000-01-06,ORCL,ORACLE CORP,94.6875,105.0000,96.0000,27848885.0,-0.058824,1423672.0,100.15625
4,2000-01-07,ORCL,ORACLE CORP,93.5625,103.5000,103.3750,23218034.0,0.076823,1423672.0,95.00000


#### CHANGE symbol, name columns to string

In [8]:
df['symbol'] = df['symbol'].astype(str)
df['name'] = df['name'].astype(str)

### CHANGE bidlo, askhi, price, vol, ret, bid, ask, sout, open to FLOAT

In [9]:
df['bidlo'] = df['bidlo'].astype(float)
df['askhi'] = df['askhi'].astype(float)
df['price'] = df['price'].astype(float)
df['vol'] = df['vol'].astype(float)
df['ret'] = df['ret'].astype(float)
#df['bid'] = df['bid'].astype(float)
#df['ask'] = df['ask'].astype(float)
df['sout'] = df['sout'].astype(float)
df['open'] = df['open'].astype(float)

#### CREATE NEW PRIMARY KEY TO BE APPLIED TO EACH TABLE
#### CONCATINATION BETWEEN SYMBOL AND COMPANY NAME

In [10]:
@timing
def create_key():
    key_list = []
    for i in range(len(df)):
        term = str(df.iloc[i, 1])
        if not term:
            value  = '0_'+df.iloc[i, 2]
            key_list.append(value)
        else:
            value = str(df.iloc[i, 1])+'_'+str(df.iloc[i, 2])
            key_list.append(value)

    return key_list

key_list = create_key()
df['key'] = key_list
del key_list

print('The final df has shape {}'.format(df.shape))
df.head()

create_key function took 78272.735 ms
The final df has shape (2330557, 11)


,date,symbol,name,bidlo,askhi,price,vol,ret,sout,open,key
0,2000-01-03,ORCL,ORACLE CORP,111.6250,125.1875,118.1250,24831819.0,0.054099,1423672.0,124.62500,ORCL_ORACLE CORP
1,2000-01-04,ORCL,ORACLE CORP,105.0000,118.6250,107.6875,29568565.0,-0.088360,1423672.0,115.50000,ORCL_ORACLE CORP
2,2000-01-05,ORCL,ORACLE CORP,96.0000,106.3750,102.0000,42032160.0,-0.052815,1423672.0,101.62500,ORCL_ORACLE CORP
3,2000-01-06,ORCL,ORACLE CORP,94.6875,105.0000,96.0000,27848885.0,-0.058824,1423672.0,100.15625,ORCL_ORACLE CORP
4,2000-01-07,ORCL,ORACLE CORP,93.5625,103.5000,103.3750,23218034.0,0.076823,1423672.0,95.00000,ORCL_ORACLE CORP


In [11]:
oknk = pd.read_csv('data\\external\\old_key_new_key.csv')
oknk.head()

,OLD_KEY,NEW_KEY
0,ACN_ACCENTURE LTD BERMUDA,ACN_ACCENTURE
1,ACN_ACCENTURE PLC IRELAND,ACN_ACCENTURE
2,ADBE_ADOBE INC,ADBE_ADOBE
3,ADBE_ADOBE SYSTEMS INC,ADBE_ADOBE
4,BIIB_BIOGEN IDEC INC,BIIB_BIOGEN


In [12]:
df_merge = pd.merge(df, oknk, left_on='key', right_on='OLD_KEY', copy=False)
df_merge = df_merge.drop_duplicates()
#df_merge.index = df_merge['NEW_KEY']
df_merge = df_merge.drop(['key'], axis=1)
df_merge.head()

,date,symbol,name,bidlo,askhi,price,vol,ret,sout,open,OLD_KEY,NEW_KEY
0,2000-01-03,ORCL,ORACLE CORP,111.6250,125.1875,118.1250,24831819.0,0.054099,1423672.0,124.62500,ORCL_ORACLE CORP,ORCL_ORACLE CORP
1,2000-01-04,ORCL,ORACLE CORP,105.0000,118.6250,107.6875,29568565.0,-0.088360,1423672.0,115.50000,ORCL_ORACLE CORP,ORCL_ORACLE CORP
2,2000-01-05,ORCL,ORACLE CORP,96.0000,106.3750,102.0000,42032160.0,-0.052815,1423672.0,101.62500,ORCL_ORACLE CORP,ORCL_ORACLE CORP
3,2000-01-06,ORCL,ORACLE CORP,94.6875,105.0000,96.0000,27848885.0,-0.058824,1423672.0,100.15625,ORCL_ORACLE CORP,ORCL_ORACLE CORP
4,2000-01-07,ORCL,ORACLE CORP,93.5625,103.5000,103.3750,23218034.0,0.076823,1423672.0,95.00000,ORCL_ORACLE CORP,ORCL_ORACLE CORP


#### CREATE LIST OF KEYS

In [13]:
# key_list = df_name.index.unique()
# print(len(key_list))
# key_list[0]

#### CREATE LIST OF NEW KEYS

In [14]:
new_key_list = df_merge['NEW_KEY'].unique()
print(len(new_key_list))
new_key_list[0]

712


'ORCL_ORACLE CORP'

In [15]:
print(df.shape)
print(df_merge.shape)

(2330557, 11)
(2330561, 12)


## CREATE TABLES/ENTITIES
###### ...
###### ...

### NAME TABLE

In [16]:
df_name = df_merge[['symbol', 'name', 'NEW_KEY']]
df_name = df_name.drop_duplicates()
df_name.index = df_name['NEW_KEY']
df_name = df_name.drop('NEW_KEY', axis=1)
df_name.to_csv('data\\interim\\df_symbols_names.csv')
df_name.head()

,symbol,name
NEW_KEY,,
ORCL_ORACLE CORP,ORCL,ORACLE CORP
MSFT_MICROSOFT CORP,MSFT,MICROSOFT CORP
TROW_T ROWE PRICE GROUP INC,TROW,T ROWE PRICE ASSOC INC
TROW_T ROWE PRICE GROUP INC,TROW,T ROWE PRICE GROUP INC
HON_HONEYWELL INTERNATIONAL INC,HON,HONEYWELL INTERNATIONAL INC


##### CHECK THE KEYS ARE UNIQUE

In [17]:
num_keys = df_name.index.value_counts().sort_values(ascending=False).sum()
num_rows = df_name.shape[0]

if num_keys == num_rows:
    print("the df are the same:\n{} unique values\n{} rows of name df".format(num_keys , num_rows))
else:
    print('the df values are not the same {} vs. {}'.format(num_keys , num_rows))
    
#df_name.index.value_counts().sort_values(ascending=False)

the df are the same:
847 unique values
847 rows of name df


## CREATE THE TABLES:
- BIDLO/BID
- ASKHI/ASK
- PRICE
- VOLUME
- RETURNS
- OPEN PRICE
- SHARES OUTSTANDING

In [18]:
# temp_list = new_key_list[:10]
# temp_list

### BIDLO

In [19]:
#[len(df[df['key']==error_bidlo[i]]) for i in range(len(error_bidlo))]

In [20]:
#df_bidlo.to_csv('data\\interim\\bidlo_temp.csv')

### 1/8/19 begin here

In [21]:
@timing
def bidlo_func():

    bidlo_error = []
    df_bidlo = df_merge[['date', 'NEW_KEY', 'bidlo']]
    ########----------------#######
    df_bidlo_temp = pd.DataFrame([])
    #for symbol in list_symbol:
    for symbol in new_key_list:
    #for symbol in error_bidlo:
        try:
            df_temp = df_bidlo[df_bidlo['NEW_KEY']==symbol] ### NO _temp IN THE FILTER
            #print('setting {} date index'.format(symbol))
            df_temp.index=df_temp['date']
            #print('subsetting {} bidlo'.format(symbol))            
            df_temp = df_temp[['bidlo']]
            #print(len(df_temp))
            #print('setting {} transpose'.format(symbol))            
            df_temp = df_temp.T
            #print('changin {} index'.format(symbol))
            df_temp.index = [symbol]
            #df_temp.rename()
            #print('CONCATENATING {}'.format(symbol))
            df_bidlo_temp = pd.concat([df_bidlo_temp, df_temp], axis=0) ### TEMP
            #print('finished with {}\n'.format(symbol))
        except Exception as e:
        #except: 
            #print(symbol, e)
            bidlo_error.append(symbol)
            
    return df_bidlo_temp, bidlo_error
    
df_bidlo, error_bidlo = bidlo_func()
print(len(error_bidlo), df_bidlo.shape)
#pd.Series(error_bidlo).to_csv('data\\interim\\error_bidlo_temp.csv')
df_bidlo.head()

bidlo_func function took 102465.452 ms
23 (689, 4780)


date,2000-01-03,2000-01-04,2000-01-05,2000-01-06,2000-01-07,2000-01-10,2000-01-11,2000-01-12,2000-01-13,2000-01-14,...,2018-12-17,2018-12-18,2018-12-19,2018-12-20,2018-12-21,2018-12-24,2018-12-26,2018-12-27,2018-12-28,2018-12-31
ORCL_ORACLE CORP,111.6250,105.0000,96.0000,94.6875,93.5625,105.5000,109.5000,103.6875,103.50000,104.750,...,45.53,45.44,45.75000,45.48,43.86,42.68,42.40,43.4600,44.74,44.84
MSFT_MICROSOFT CORP,112.0000,112.2500,109.3750,108.3750,107.3125,111.3750,108.6875,104.4375,101.50000,105.750,...,101.71,102.52,101.35000,98.78,97.46,93.98,93.96,96.4000,99.52,100.44
TROW_T ROWE PRICE GROUP INC,34.6250,34.0000,33.1875,33.3750,34.5000,34.8750,34.5625,35.1875,36.59375,38.375,...,90.69,90.33,88.07000,86.91,86.39,84.79,84.59,88.0000,90.68,91.33
HON_HONEYWELL INTERNATIONAL INC,56.5000,54.0625,54.6875,54.5000,56.4375,58.3125,58.7500,59.0625,59.50000,59.250,...,133.58,134.38,132.14999,129.37,129.33,124.63,123.48,126.8438,130.25,130.47
T_A T & T CORP,50.5625,51.2500,50.6250,47.5000,47.8125,49.5625,51.0625,51.6875,53.18750,53.625,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Error Note
- There are Duplicate dates for each stock in the error dataframe


#### Take LOWEST BIDLO value for each of the duplicated dates
- Same with BID

In [22]:
df_error_bidlo = pd.DataFrame([])

for error_comapny in error_bidlo:
    temp_df = df_merge[df_merge['NEW_KEY']==error_comapny]
    temp_df = temp_df[['NEW_KEY', 'date', 'bidlo']]
    df_error_bidlo = pd.concat([df_error_bidlo, temp_df])

df_error_bidlo = df_error_bidlo.sort_values(['NEW_KEY', 'date'])

print('shape before drop: {}'.format(df_error_bidlo.shape))
df_error_bidlo = df_error_bidlo.groupby(['NEW_KEY', 'date'], sort=False).min()
print('shape after the drop {}'.format(df_error_bidlo.shape))
df_error_bidlo.head()

##########---------------###-------########------------#########

print('before adding error dataframes: {}'.format(df_bidlo.shape))

for symbol in error_bidlo:
    df_temp = df_error_bidlo.loc[symbol] ### NO _temp IN THE FILTER
    df_temp = df_temp.T
    df_temp.index = [symbol]
    df_bidlo = pd.concat([df_bidlo, df_temp], axis=0) ### TEMP
    
print('after adding error dataframes: {}'.format(df_bidlo.shape))
df_bidlo.head()

shape before drop: (103535, 3)
shape after the drop (76938, 1)
before adding error dataframes: (689, 4780)
after adding error dataframes: (712, 4780)


date,2000-01-03,2000-01-04,2000-01-05,2000-01-06,2000-01-07,2000-01-10,2000-01-11,2000-01-12,2000-01-13,2000-01-14,...,2018-12-17,2018-12-18,2018-12-19,2018-12-20,2018-12-21,2018-12-24,2018-12-26,2018-12-27,2018-12-28,2018-12-31
ORCL_ORACLE CORP,111.6250,105.0000,96.0000,94.6875,93.5625,105.5000,109.5000,103.6875,103.50000,104.750,...,45.53,45.44,45.75000,45.48,43.86,42.68,42.40,43.4600,44.74,44.84
MSFT_MICROSOFT CORP,112.0000,112.2500,109.3750,108.3750,107.3125,111.3750,108.6875,104.4375,101.50000,105.750,...,101.71,102.52,101.35000,98.78,97.46,93.98,93.96,96.4000,99.52,100.44
TROW_T ROWE PRICE GROUP INC,34.6250,34.0000,33.1875,33.3750,34.5000,34.8750,34.5625,35.1875,36.59375,38.375,...,90.69,90.33,88.07000,86.91,86.39,84.79,84.59,88.0000,90.68,91.33
HON_HONEYWELL INTERNATIONAL INC,56.5000,54.0625,54.6875,54.5000,56.4375,58.3125,58.7500,59.0625,59.50000,59.250,...,133.58,134.38,132.14999,129.37,129.33,124.63,123.48,126.8438,130.25,130.47
T_A T & T CORP,50.5625,51.2500,50.6250,47.5000,47.8125,49.5625,51.0625,51.6875,53.18750,53.625,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### ASKHI TABLE

In [23]:
@timing
def askhi_func():
    error_askhi = []
    df_askhi = df_merge[['date', 'NEW_KEY', 'askhi']]
    df_askhi_temp = pd.DataFrame([])
    
    for symbol in new_key_list:
        try:
            df_temp = df_askhi[df_askhi['NEW_KEY']==symbol]
            df_temp.index=df_temp['date']
            df_temp = df_temp[['askhi']]
            df_temp = df_temp.T
            df_temp.index = [symbol]
            df_askhi_temp = pd.concat([df_askhi_temp, df_temp], axis=0)

        #except:
        except Exception as e:
            #print(symbol, e)
            error_askhi.append(symbol)
    return df_askhi_temp, error_askhi

df_askhi, error_askhi = askhi_func()
print(len(error_askhi), df_askhi.shape)
#pd.Series(error_askhi).to_csv('..\data\\error_askhi.csv')
#df_askhi.head(2)

askhi_func function took 100273.540 ms
23 (689, 4780)


### Take the error dataframe and filter out the highest ASKHI value for each of the days of each of the companies
- Same with ASK

In [24]:
df_error_askhi = pd.DataFrame([])

for error_comapny in error_askhi:
    temp_df = df_merge[df_merge['NEW_KEY']==error_comapny]
    temp_df = temp_df[['NEW_KEY', 'date', 'askhi']]
    df_error_askhi = pd.concat([df_error_askhi, temp_df])

df_error_askhi = df_error_askhi.sort_values(['NEW_KEY', 'date'])

print('shape before drop: {}'.format(df_error_askhi.shape))
"""TAKE THE MAX ASKHI"""
df_error_askhi = df_error_askhi.groupby(['NEW_KEY', 'date'], sort=False).max()
print('shape after the drop {}'.format(df_error_askhi.shape))
#df_error_askhi.head()

#######----------#############-------------########
print('before adding error dataframes: {}'.format(df_askhi.shape))

for symbol in error_askhi:
    df_temp = df_error_askhi.loc[symbol] ### NO _temp IN THE FILTER
    df_temp = df_temp.T
    df_temp.index = [symbol]
    df_askhi = pd.concat([df_askhi, df_temp], axis=0) ### TEMP
    
print('after adding error dataframes: {}'.format(df_askhi.shape))
df_askhi.head()

shape before drop: (103535, 3)
shape after the drop (76938, 1)
before adding error dataframes: (689, 4780)
after adding error dataframes: (712, 4780)


date,2000-01-03,2000-01-04,2000-01-05,2000-01-06,2000-01-07,2000-01-10,2000-01-11,2000-01-12,2000-01-13,2000-01-14,...,2018-12-17,2018-12-18,2018-12-19,2018-12-20,2018-12-21,2018-12-24,2018-12-26,2018-12-27,2018-12-28,2018-12-31
ORCL_ORACLE CORP,125.1875,118.6250,106.3750,105.0000,103.500,116.0000,114.7500,112.250,109.8750,111.3750,...,47.23000,48.34000,47.44000,46.95,46.21,43.82000,44.61000,45.09,45.76500,45.50
MSFT_MICROSOFT CORP,118.6250,117.1250,116.3750,113.8750,112.250,113.6875,114.2500,108.875,108.6250,113.9375,...,105.80000,104.51000,106.88000,104.31,103.00,97.97000,100.69000,101.19,102.41000,102.40
TROW_T ROWE PRICE GROUP INC,37.2500,35.1250,34.9375,35.0000,35.500,35.7500,35.5625,36.500,39.6875,39.1250,...,93.32520,92.64000,92.12000,88.99,90.33,87.53250,89.73000,91.37,92.42000,92.69
HON_HONEYWELL INTERNATIONAL INC,58.5000,56.4375,56.2500,56.1250,59.375,59.8750,60.1875,59.750,60.0000,60.5000,...,137.17999,136.75999,137.75999,133.62,132.45,128.99001,129.60001,131.63,133.07001,132.16
T_A T & T CORP,53.4375,53.2500,52.3750,50.8125,49.875,51.6875,52.0000,54.625,56.0000,55.0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### PRICE TABLE

In [25]:
@timing
def price_func():
    error_price = []
    
    df_price = df_merge[['date', 'NEW_KEY', 'price']]
    ########----------------#######
    df_price_temp = pd.DataFrame([])
    for symbol in new_key_list:
        try:
            df_temp = df_price[df_price['NEW_KEY']==symbol] ### NO _temp IN THE FILTER
            df_temp.index=df_temp['date']
            df_temp = df_temp[['price']]
            df_temp = df_temp.T
            df_temp.index = [symbol]
            #df_temp.rename()
            df_price_temp = pd.concat([df_price_temp, df_temp], axis=0) ### TEMP
        except Exception as e:
            #print(symbol, e)
            error_price.append(symbol)
    return df_price_temp, error_price

df_price, error_price = price_func()
print(len(error_price), df_price.shape)
#pd.Series(error_price).to_csv('..\data\\error_price.csv')
#df_price.head(2)

price_func function took 105428.513 ms
23 (689, 4780)


##### take MAX price

In [26]:
df_error_price = pd.DataFrame([])

for error_comapny in error_price:
    temp_df = df_merge[df_merge['NEW_KEY']==error_comapny]
    temp_df = temp_df[['NEW_KEY', 'date', 'askhi']]
    df_error_price = pd.concat([df_error_price, temp_df])

df_error_price = df_error_price.sort_values(['NEW_KEY', 'date'])

print('shape before drop: {}'.format(df_error_price.shape))
"""TAKE THE MAX PRICE"""
df_error_price = df_error_price.groupby(['NEW_KEY', 'date'], sort=False).max()
print('shape after the drop {}'.format(df_error_price.shape))
#df_error_askhi.head()

#######----------#############-------------########
print('before adding error dataframes: {}'.format(df_price.shape))

for symbol in error_askhi:
    df_temp = df_error_price.loc[symbol] ### NO _temp IN THE FILTER
    df_temp = df_temp.T
    df_temp.index = [symbol]
    df_price = pd.concat([df_price, df_temp], axis=0) ### TEMP

    
"""DROP DUPLICATES"""
df_price = df_price.drop_duplicates()    
print('after adding error dataframes: {}'.format(df_price.shape))
df_price.head()

shape before drop: (103535, 3)
shape after the drop (76938, 1)
before adding error dataframes: (689, 4780)
after adding error dataframes: (712, 4780)


date,2000-01-03,2000-01-04,2000-01-05,2000-01-06,2000-01-07,2000-01-10,2000-01-11,2000-01-12,2000-01-13,2000-01-14,...,2018-12-17,2018-12-18,2018-12-19,2018-12-20,2018-12-21,2018-12-24,2018-12-26,2018-12-27,2018-12-28,2018-12-31
ORCL_ORACLE CORP,118.1250,107.6875,102.0000,96.0,103.3750,115.7500,112.375,105.6250,105.0625,106.8125,...,45.73,45.85000,46.45000,46.24,44.00,42.69,44.59000,44.97000,44.82000,45.15
MSFT_MICROSOFT CORP,116.5625,112.6250,113.8125,110.0,111.4375,112.2500,109.375,105.8125,107.8125,112.2500,...,102.89,103.97000,103.69000,101.51,98.23,94.13,100.56000,101.18000,100.39000,101.57
TROW_T ROWE PRICE GROUP INC,35.1250,34.0625,34.0000,35.0,34.7500,35.1250,35.000,36.5000,38.5000,39.0625,...,91.03,90.80000,88.41000,87.30,86.78,84.95,89.65000,91.31000,91.34000,92.32
HON_HONEYWELL INTERNATIONAL INC,56.6875,55.6875,54.9375,56.0,59.0000,59.3125,58.875,59.4375,59.8750,59.8750,...,134.38,135.17999,133.25999,130.67,129.44,124.83,129.57001,131.57001,130.75999,132.12
T_A T & T CORP,53.3750,51.2500,50.9375,48.5,49.0000,50.8125,51.250,54.6250,53.3750,54.6875,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### VOLUME TABLE

In [27]:
@timing
def volume_func():
    error_volume = []
    
    df_volume = df_merge[['date', 'NEW_KEY', 'vol']]
    ########----------------#######
    df_volume_temp = pd.DataFrame([])
    for symbol in new_key_list:
        try:            
            df_temp = df_volume[df_volume['NEW_KEY']==symbol] ### NO _temp IN THE FILTER
            df_temp.index=df_temp['date']
            df_temp = df_temp[['vol']]
            df_temp = df_temp.T
            df_temp.index = [symbol]
            #df_temp.rename()
            df_volume_temp = pd.concat([df_volume_temp, df_temp], axis=0) ### TEMP    
        except:
            error_volume.append(symbol)
    return df_volume_temp, error_volume

df_volume, error_volume = volume_func()
print(len(error_volume), df_volume.shape)
#pd.Series(error_volume).to_csv('..\data\\error_volume.csv')
#df_volume.head(2)        

volume_func function took 111525.708 ms
23 (689, 4780)


##### take MAX volume

In [28]:
df_error_volume = pd.DataFrame([])

for error_comapny in error_volume:
    temp_df = df_merge[df_merge['NEW_KEY']==error_comapny]
    temp_df = temp_df[['NEW_KEY', 'date', 'vol']]
    df_error_volume = pd.concat([df_error_volume, temp_df])

df_error_volume = df_error_volume.sort_values(['NEW_KEY', 'date'])

print('shape before drop: {}'.format(df_error_volume.shape))
"""TAKE THE MAX PRICE"""
df_error_volume = df_error_volume.groupby(['NEW_KEY', 'date'], sort=False).max()
print('shape after the drop {}'.format(df_error_volume.shape))
#df_error_askhi.head()

#######----------#############-------------########
print('before adding error dataframes: {}'.format(df_volume.shape))

for symbol in error_volume:
    df_temp = df_error_volume.loc[symbol] ### NO _temp IN THE FILTER
    df_temp = df_temp.T
    df_temp.index = [symbol]
    df_volume = pd.concat([df_volume, df_temp], axis=0) ### TEMP

    
"""DROP DUPLICATES"""
df_volume = df_volume.drop_duplicates()    
print('after adding error dataframes: {}'.format(df_volume.shape))
df_volume.head()

shape before drop: (103535, 3)
shape after the drop (76938, 1)
before adding error dataframes: (689, 4780)
after adding error dataframes: (712, 4780)


date,2000-01-03,2000-01-04,2000-01-05,2000-01-06,2000-01-07,2000-01-10,2000-01-11,2000-01-12,2000-01-13,2000-01-14,...,2018-12-17,2018-12-18,2018-12-19,2018-12-20,2018-12-21,2018-12-24,2018-12-26,2018-12-27,2018-12-28,2018-12-31
ORCL_ORACLE CORP,24831819.0,29568565.0,42032160.0,27848885.0,23218034.0,23630411.0,21910101.0,21110284.0,14255122.0,14531999.0,...,30441145.0,42855600.0,34397250.0,35165179.0,58768410.0,17230381.0,21428929.0,19048702.0,39141279.0,14932651.0
MSFT_MICROSOFT CORP,26937123.0,27531141.0,32708345.0,27997348.0,31415665.0,24322885.0,23857503.0,33588432.0,42115412.0,37321270.0,...,56956564.0,49417797.0,68187880.0,70334184.0,111239770.0,43935112.0,51627193.0,49495409.0,38162485.0,33173365.0
TROW_T ROWE PRICE GROUP INC,381616.0,545646.0,535862.0,252966.0,237955.0,207805.0,210289.0,364494.0,886817.0,519978.0,...,1959973.0,1804473.0,1863185.0,2156700.0,4232675.0,1005519.0,1587486.0,1290839.0,1093473.0,1449392.0
HON_HONEYWELL INTERNATIONAL INC,2100799.0,2670299.0,2975699.0,2388000.0,3943399.0,1467399.0,2641899.0,2046599.0,1543899.0,2549000.0,...,3133214.0,2637752.0,3868924.0,4518936.0,6903728.0,2838791.0,3440398.0,3249638.0,2759324.0,2199898.0
T_A T & T CORP,14720799.0,8905000.0,11428299.0,9050099.0,13844500.0,15874299.0,12534500.0,16481000.0,14353399.0,9707899.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### RETURNS TABLE

In [29]:
@timing
def returns_func():
    error_returns = []
    df_return = df_merge[['date', 'NEW_KEY', 'ret']]
    ########----------------#######
    df_return_temp = pd.DataFrame([])
    for symbol in new_key_list:
        try:
            df_temp = df_return[df_return['NEW_KEY']==symbol] ### NO _temp IN THE FILTER
            df_temp.index=df_temp['date']
            df_temp = df_temp[['ret']]
            df_temp = df_temp.T
            df_temp.index = [symbol]
            #df_temp.rename()
            df_return_temp = pd.concat([df_return_temp, df_temp], axis=0) ### TEMP
        except:
            error_returns.append(symbol)
    return df_return_temp, error_returns

df_returns, error_returns = returns_func()
print(len(error_returns), df_returns.shape)
#pd.Series(error_returns).to_csv('..\data\\error_returns.csv')
#df_returns.head(2)            

returns_func function took 106097.471 ms
23 (689, 4780)


##### take max RETUNRS 

In [30]:
df_error_returns = pd.DataFrame([])

for error_comapny in error_returns:
    temp_df = df_merge[df_merge['NEW_KEY']==error_comapny]
    temp_df = temp_df[['NEW_KEY', 'date', 'vol']]
    df_error_returns = pd.concat([df_error_returns, temp_df])

df_error_returns = df_error_returns.sort_values(['NEW_KEY', 'date'])

print('shape before drop: {}'.format(df_error_returns.shape))
"""TAKE THE MAX PRICE"""
df_error_returns = df_error_returns.groupby(['NEW_KEY', 'date'], sort=False).max()
print('shape after the drop {}'.format(df_error_returns.shape))
#df_error_askhi.head()

#######----------#############-------------########
print('before adding error dataframes: {}'.format(df_returns.shape))

for symbol in error_returns:
    df_temp = df_error_returns.loc[symbol] ### NO _temp IN THE FILTER
    df_temp = df_temp.T
    df_temp.index = [symbol]
    df_returns = pd.concat([df_returns, df_temp], axis=0) ### TEMP

    
"""DROP DUPLICATES"""
df_returns = df_returns.drop_duplicates()    
print('after adding error dataframes: {}'.format(df_returns.shape))
df_returns.head()

shape before drop: (103535, 3)
shape after the drop (76938, 1)
before adding error dataframes: (689, 4780)
after adding error dataframes: (712, 4780)


date,2000-01-03,2000-01-04,2000-01-05,2000-01-06,2000-01-07,2000-01-10,2000-01-11,2000-01-12,2000-01-13,2000-01-14,...,2018-12-17,2018-12-18,2018-12-19,2018-12-20,2018-12-21,2018-12-24,2018-12-26,2018-12-27,2018-12-28,2018-12-31
ORCL_ORACLE CORP,0.054099,-0.088360,-0.052815,-0.058824,0.076823,0.119710,-0.029158,-0.060067,-0.005325,0.016657,...,-0.018670,0.002624,0.013086,-0.004521,-0.048443,-0.029773,0.044507,0.008522,-0.003336,0.007363
MSFT_MICROSOFT CORP,-0.001606,-0.033780,0.010544,-0.033498,0.013068,0.007291,-0.025612,-0.032571,0.018901,0.041159,...,-0.029614,0.010497,-0.002693,-0.021024,-0.032312,-0.041739,0.068310,0.006166,-0.007808,0.011754
TROW_T ROWE PRICE GROUP INC,-0.049069,-0.030249,-0.001835,0.029412,-0.007143,0.010791,-0.003559,0.042857,0.054795,0.014610,...,-0.010221,-0.002527,-0.026322,-0.012555,-0.005957,-0.021088,0.055327,0.018516,0.000329,0.010729
HON_HONEYWELL INTERNATIONAL INC,-0.017335,-0.017641,-0.013468,0.019340,0.053571,0.005297,-0.007376,0.009554,0.007361,0.000000,...,-0.015098,0.005953,-0.014203,-0.019436,-0.009413,-0.035615,0.037972,0.015436,-0.006157,0.010401
T_A T & T CORP,0.050431,-0.039813,-0.006098,-0.047853,0.010309,0.036990,0.008610,0.065854,-0.022883,0.024590,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### SHARES OUTSTANDING TABLE

In [31]:
@timing
def shares_out_func():
    error_shares_out = []
    df_shares_out = df_merge[['date', 'NEW_KEY', 'sout']]
    ########----------------#######
    df_shares_out_temp = pd.DataFrame([])
    for symbol in new_key_list:
        try:
            df_temp = df_shares_out[df_shares_out['NEW_KEY']==symbol] ### NO _temp IN THE FILTER
            df_temp.index=df_temp['date']
            df_temp = df_temp[['sout']] 
            df_temp = df_temp.T
            df_temp.index = [symbol]
            #df_temp.rename()
            df_shares_out_temp = pd.concat([df_shares_out_temp, df_temp], axis=0) ### TEMP    
        except:
            error_shares_out.append(symbol)
    return df_shares_out_temp, error_shares_out

df_shares_out, error_shares_out = shares_out_func()
print(len(error_shares_out), df_shares_out.shape)
#pd.Series(error_shares_out).to_csv('..\data\\error_shares_out.csv')
#df_shares_out.head(2)                  

shares_out_func function took 99384.376 ms
23 (689, 4780)


### take MAX shares outstanding

In [32]:
df_error_shares_out = pd.DataFrame([])

for error_comapny in error_returns:
    temp_df = df_merge[df_merge['NEW_KEY']==error_comapny]
    temp_df = temp_df[['NEW_KEY', 'date', 'vol']]
    df_error_shares_out = pd.concat([df_error_shares_out, temp_df])

df_error_shares_out = df_error_shares_out.sort_values(['NEW_KEY', 'date'])

print('shape before drop: {}'.format(df_error_shares_out.shape))
"""TAKE THE MAX PRICE"""
df_error_shares_out = df_error_shares_out.groupby(['NEW_KEY', 'date'], sort=False).max()
print('shape after the drop {}'.format(df_error_shares_out.shape))
#df_error_askhi.head()

#######----------#############-------------########
print('before adding error dataframes: {}'.format(df_shares_out.shape))

for symbol in error_returns:
    df_temp = df_error_shares_out.loc[symbol] ### NO _temp IN THE FILTER
    df_temp = df_temp.T
    df_temp.index = [symbol]
    df_shares_out = pd.concat([df_shares_out, df_temp], axis=0) ### TEMP

    
"""DROP DUPLICATES"""
df_shares_out = df_shares_out.drop_duplicates()    
print('after adding error dataframes: {}'.format(df_shares_out.shape))
df_shares_out.head()

shape before drop: (103535, 3)
shape after the drop (76938, 1)
before adding error dataframes: (689, 4780)
after adding error dataframes: (712, 4780)


date,2000-01-03,2000-01-04,2000-01-05,2000-01-06,2000-01-07,2000-01-10,2000-01-11,2000-01-12,2000-01-13,2000-01-14,...,2018-12-17,2018-12-18,2018-12-19,2018-12-20,2018-12-21,2018-12-24,2018-12-26,2018-12-27,2018-12-28,2018-12-31
ORCL_ORACLE CORP,1423672.0,1423672.0,1423672.0,1423672.0,1423672.0,1423672.0,1423672.0,1423672.0,1423672.0,1423672.0,...,3787960.0,3787960.0,3787960.0,3787960.0,3787960.0,3787960.0,3787960.0,3787960.0,3787960.0,3588919.0
MSFT_MICROSOFT CORP,5160025.0,5160025.0,5160025.0,5160025.0,5160025.0,5160025.0,5160025.0,5160025.0,5160025.0,5160025.0,...,7676219.0,7676219.0,7676219.0,7676219.0,7676219.0,7676219.0,7676219.0,7676219.0,7676219.0,7676219.0
TROW_T ROWE PRICE GROUP INC,120678.0,120678.0,120678.0,120678.0,120678.0,120678.0,120678.0,120678.0,120678.0,120678.0,...,240655.0,240655.0,240655.0,240655.0,240655.0,240655.0,240655.0,240655.0,240655.0,240655.0
HON_HONEYWELL INTERNATIONAL INC,789233.0,789233.0,789233.0,789233.0,789233.0,789233.0,789233.0,789233.0,789233.0,789233.0,...,740288.0,740288.0,740288.0,740288.0,740288.0,740288.0,740288.0,740288.0,740288.0,740288.0
T_A T & T CORP,3195346.0,3195346.0,3195346.0,3195346.0,3195346.0,3195346.0,3195346.0,3195346.0,3195346.0,3195346.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### BID/ASK SPREAD TABLE

In [33]:
df_spread = df_askhi - df_bidlo

### MARKET CAP TABLE

In [34]:
df_market_cap = df_shares_out*df_price

##### GENERATE LIST OF DATAFRAMES

In [35]:
df_list = [df_bidlo, df_askhi, df_price, df_volume, df_returns, df_shares_out, df_spread, df_market_cap]
new_df_list = []
df_name_list = ['df_bidlo', 'df_askhi', 'df_price', 'df_volume', 'df_returns', 'df_shares_out', 'df_spread', 'df_market_cap']

### There was a duplicte row value for JCI_JOHNSON CONTROLS INTL PLC, so we dropped it

In [36]:
for i in range(len(df_list)):
    print(df_name_list[i])
    temp_value = df_list[i].loc['JCI_JOHNSON CONTROLS INTL PLC', '2016-09-06'][1]
    df_list[i].loc['JCI_JOHNSON CONTROLS INTL PLC', '2016-09-06'] = temp_value
    
    temp_df = df_list[i].T.drop_duplicates()
    #print(temp_df.head())
    new_df_list.append(temp_df.T)

del df_list

df_bidlo
df_askhi
df_price
df_volume
df_returns
df_shares_out
df_spread
df_market_cap


In [37]:
#new_df_list[0]

In [38]:
#del df_bidlo

In [39]:
#os.remove('data\\processed\\df_bidlo')

### UPLOAD THE DATA INTO THE S3 BUCKET

##### READ CREDENTIALS TEXT FILE
##### CREDENTIALS BELONG TO "capstoner" user role in AWS

In [40]:
import boto3
from botocore.client import Config
import os
#os.chdir('C:\\Users\\A Sua\\Documents\\AWS\\credentials')
list_of_lines = []
with open('C:\\Users\\A Sua\\Documents\\AWS\\credentials\\capstoner_key.txt') as fp:
    lines = fp.readlines()
    for line in lines:
        line = line.strip()
        list_of_lines.append(line)
    fp.close()

location = list_of_lines[0].split('=')[1]
AWS_ACCESS_KEY_ID = list_of_lines[1].split('=')[1]
AWS_SECRET_ACCESS_KEY = list_of_lines[2].split('=')[1]
BUCKET_NAME = 'capstonefiu2020-data'

In [41]:
@timing
def upload_PENN_to_aws():
    for i in range(len(new_df_list)):
        path_to_df = 'data\\processed\\{}.csv'.format(df_name_list[i])
        new_df_list[i].to_csv(path_to_df)
        
        ## open file
        temp_data = open(path_to_df, 'rb')
        
        ### instantiate s3 client
        s3 = boto3.resource('s3', 
                  aws_access_key_id = AWS_ACCESS_KEY_ID,
                   aws_secret_access_key=AWS_SECRET_ACCESS_KEY)
        s3.Bucket(BUCKET_NAME).put_object(Key='data/processed/{}.csv'.format(df_name_list[i]), Body=temp_data)
        
        ### close the file
        temp_data.close()
        os.remove(path_to_df)
        print('{} added to cloud'.format(df_name_list[i]))
    print('finished uploading dataframes')
    
upload_PENN_to_aws()

finished uploading dataframes
upload_PENN_to_aws function took 228127.394 ms


In [42]:
# ## FROM capstoner user role
# data = open('C:\\Users\\A Sua\\Documents\\AWS\\credentials\\test_csv.csv', 'rb')

# s3 = boto3.resource('s3', 
#               aws_access_key_id = AWS_ACCESS_KEY_ID,
#                aws_secret_access_key=AWS_SECRET_ACCESS_KEY)


# s3.Bucket(BUCKET_NAME).put_object(Key='data/test_csv.csv', Body=data)

# data.close()

In [43]:
# for i in range(len(df_list)):
#     print(df_name_list[i])
#     print(new_df_list[i].columns.value_counts()[:2])

In [44]:
# df_askhi.loc['JCI_JOHNSON CONTROLS INTL PLC', '2016-09-06'] = 48.97
# df_askhi.loc['JCI_JOHNSON CONTROLS INTL PLC', '2016-09-06']

In [45]:
# df_askhi.T.drop_duplicates().loc['2016-09-06', 'JCI_JOHNSON CONTROLS INTL PLC']

In [46]:
# df_bidlo.loc['JCI_JOHNSON CONTROLS INTL PLC', '2016-09-06']
# #df_bidlo.loc['JCI_JOHNSON CONTROLS INTL PLC', '2016-09-06']


In [47]:
# for i in range(len(df_list)):
#     print(df_name_list[i])
#     temp_value = df_list[i].loc['JCI_JOHNSON CONTROLS INTL PLC', '2016-09-06'][1]
#     df_list[i].loc['JCI_JOHNSON CONTROLS INTL PLC', '2016-09-06'] = temp_value
    
#     temp_df = df_list[i].T.drop_duplicates()
#     #print(temp_df.head())
#     new_df_list.append(temp_df.T)

### Take the error dataframe and filter out the highest RETURN highest return if >=0 else return the lowest return if <0 
- Same with return

### SHARES OUTSTANDING take the HIGHEST

# WORKSPACE

#### what is the date range for the longest trading stock?

#### how many different company names are there?
#### note: there is the possibility that a symbol can have multiple company names
#### which companies lasted through the 20-year period?
#### 

### each of the companies has a different number of rows 
### so what we have to do is make a skeleton table and merge it with the original dataframe on a Left join

### DONT FORGET TO ADD THE MAX SPREAD (ASK-BID)

### BID TABLE

In [48]:
# @timing
# def bid_func():

#     error_bid = []
#     df_bid = df[['date', 'symbol', 'bid']]
#     df_bid_temp = pd.DataFrame([])
#     for symbol in list_symbol:
#         try:        
#             df_temp = df_bid[df_bid['symbol']==symbol]
#             df_temp.index=df_temp['date']
#             df_temp = df_temp[['bid']]
#             df_temp = df_temp.T
#             df_temp.index = [symbol]
#             #df_temp.rename()
#             df_bid_temp = pd.concat([df_bid_temp, df_temp], axis=0)
#         except:
#             error_bid.append(symbol)
#     return df_bid_temp, error_bid

# df_bid, error_bid = bid_func()
# print(len(error_bid))
# pd.Series(error_bid).to_csv('..\data\\error_bid.csv')
# df_bid.head(2)

### ASK TABLE

In [49]:
# @timing
# def ask_func():
#     error_ask = []

#     df_ask = df[['date', 'symbol', 'ask']]
#     ########----------------#######
#     df_ask_temp = pd.DataFrame([])
#     for symbol in list_symbol:
#         try:            
#             df_temp = df_ask[df_ask['symbol']==symbol] ### NO _temp IN THE FILTER
#             df_temp.index=df_temp['date']
#             df_temp = df_temp[['ask']]
#             df_temp = df_temp.T
#             df_temp.index = [symbol]
#             #df_temp.rename()
#             df_ask_temp = pd.concat([df_ask_temp, df_temp], axis=0)

#         except:
#             error_ask.append(symbol)
#     return df_ask_temp, error_ask

# df_ask, error_ask = ask_func()
# print(len(error_ask))
# pd.Series(error_ask).to_csv('..\data\\error_ask.csv')
# df_ask.head(2)

### OPEN PRICE TABLE

In [50]:
# @timing
# def open_func():
#     error_open = []
#     df_open = df[['date', 'symbol', 'open']]
#     ########----------------#######
#     df_open_temp = pd.DataFrame([])
#     for symbol in list_symbol:
#         try:            
#             df_temp = df_open[df_open['symbol']==symbol] ### NO _temp IN THE FILTER
#             df_temp.index=df_temp['date']
#             df_temp = df_temp[['open']] 
#             df_temp = df_temp.T
#             df_temp.index = [symbol]
#             #df_temp.rename()
#             df_open_temp = pd.concat([df_open_temp, df_temp], axis=0) ### TEM
#         except:
#             error_open.append(symbol)
#     return df_open_temp, error_open

# df_open, error_open = open_func()
# print(len(error_open))
# pd.Series(error_open).to_csv('..\data\\error_open.csv')
# df_open.head(2)                    

In [51]:
# df_askhi.T.index.value_counts().to_csv('data\\external\\temp_dates.csv')

In [52]:
# df_merge['date'].value_counts().to_csv('data\\external\\temp_dates.csv')